---
# **Big Data Analytics with Python**
 

### <span style="color:red">Note: all datasets where shared </span>
---

# **Part 1: Reading, Writing and Validating Data in PySpark HW**

Welcome to your first coding homework assignment in PySpark! I hope you enjoyed the lecture on Reading, Writing and Validating dataframes. Now it's time to put what you've learned into action! 

I've included several instructions below to help guide you through this homework assignment which I hope will get you feeling even comfortable reading, writing and validating dataframes. If you get stuck at any point, feel free to jump to the next lecture where I will guide you through my solutions to the HW assignment. 

Have fun!

Let's dig right in!


## But first things first.....
We need to always begin every Spark session by creating a Spark instance. Let's go ahead and use the method we learned in the lecture in the cell below. Also see if you can remember how to open the Spark UI (using a link that automatically guides you there). 

In [1]:
# PySpark instance

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("Joseph_wrk1").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
appid = spark._jsc.sc().applicationId()
print("You are working with", cores, "core(s) on appid: ",appid)
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/19 13:10:42 WARN Utils: Your hostname, redsteam-inspiron-3580, resolves to a loopback address: 127.0.1.1; using 10.6.247.134 instead (on interface wlp3s0)
25/11/19 13:10:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/19 13:10:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


You are working with 1 core(s) on appid:  local-1763557851399


## Next let's start by reading a basic csv dataset

Download the pga_tour_historical dataset that is attached to this lecture and save it whatever folder you want, then read it in. 

**Data Source:** https://www.kaggle.com/bradklassen/pga-tour-20102018-data

Rememer to try letting Spark infer the header and infer the Schema types!

In [ ]:
path = "data-sets/"

df_pga = spark.read.csv(path + 'pga_tour_historical.csv', header=True, inferSchema=True)
# df_pga.count()
# df_pga.printSchema()
#df_pga.limit(5)

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: string (nullable = true)



## 1. View first 5 lines of dataframe
First generate a view of the first 5 lines of the dataframe to get an idea of what is inside. We went over two ways of doing this... see if you can remember BOTH ways. 

In [51]:
# M'thd 1
df_pga.show(5)

# M'thd 2
df_pga.limit(5).toPandas()
 

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
+---------------+------+----------------+--------------------+-----+
only showing top 5 rows


,Player Name,Season,Statistic,Variable,Value
0,Robert Garrigus,2010,Driving Distance,Driving Distance - (ROUNDS),71
1,Bubba Watson,2010,Driving Distance,Driving Distance - (ROUNDS),77
2,Dustin Johnson,2010,Driving Distance,Driving Distance - (ROUNDS),83
3,Brett Wetterich,2010,Driving Distance,Driving Distance - (ROUNDS),54
4,J.B. Holmes,2010,Driving Distance,Driving Distance - (ROUNDS),100


## 2. Print the schema details

Now print the details of the dataframes schema that Spark infered to ensure that it was infered correctly. Sometimes it is not infered correctly, so we need to watch out!

In [52]:
df_pga.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: string (nullable = true)



## 3. Edit the schema during the read in

We can see from the output above that Spark did not correctly infer that the "value" column was an integer value. Let's try specifying the schema this time to let spark know what the schema should be.

Here is a link to see a list of PySpark data types in case you need it (also attached to the lecture): 
https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [53]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Custom schema
pga_schema = StructType([
    StructField("Player Name", StringType(), True),
    StructField("Season", IntegerType(), True),
    StructField("Statistic", StringType(), True),
    StructField("Variable", StringType(), True),
    StructField("Value", IntegerType(), True) 
])


 # Re-read
df_pga = spark.read.csv(path + 'pga_tour_historical.csv', header=True, schema=pga_schema)


df_pga.printSchema()

df_pga.show(5)

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: integer (nullable = true)

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
+---------------+------+----------------+--------------------+-----+
only showing top 5 rows


## 4. Generate summary statistics for only one variable

See if you can generate summary statistics for only the "Value" column using the .describe function

(count, mean, stddev, min, max) 

In [54]:
df_pga.describe("Value").show()

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|           1657247|
|   mean|12494.388998743096|
| stddev| 157274.7567357075|
|    min|              -178|
|    max|           3564954|
+-------+------------------+



## 5. Generate summary statistics for TWO variables
Now try to generate ONLY the count min and max for BOTH the "Value" and "Season" variable using the select. You can't use the .describe function for this one but see if you can remember which function you CAN use. 

In [55]:
df_pga.select("Value", "Season").summary("count", "min", "max").show()

+-------+-------+-------+
|summary|  Value| Season|
+-------+-------+-------+
|  count|1657247|2740403|
|    min|   -178|   2010|
|    max|3564954|   2018|
+-------+-------+-------+



## 6. Write a parquet file

Now try writing a parquet file (not partitioned) from the pga dataset. But first create a new dataframe containing ONLY the the "Season" and "Value" fields (using the "select command you used in the question above) and write a parquet file partitioned by "Season". This is a bit of a challenge aimed at getting you ready for material that will be covered later on in the course. Don't feel bad if you can't figure it out.

*Note that if any of your variable names contain spaces, spark will produce an error message with this call. That is why we are selecting ONLY the "Season" and "Value" fields. Ideally we should renamed those columns but we haven't gotten to that yet in this course but we will soon!*

In [56]:
# DataFrame "Season" and "Value" as sv
df_pga_sv = df_pga.select("Season", "Value")
df_pga_sv.toPandas()

,Season,Value
0,2010,71.0
1,2010,77.0
2,2010,83.0
3,2010,54.0
4,2010,100.0
...,...,...
2740398,2018,NaN
2740399,2018,NaN
2740400,2018,NaN
2740401,2018,NaN


In [57]:
# Non-partitioned parquet
df_pga_sv.write.mode("overwrite").parquet("pga_non_partitioned_parquet")

25/11/15 21:34:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


## 7. Write a partioned parquet file

You will need to use the same limited dataframe that you created in the previous question to accomplish this task as well. 

In [58]:
df_pga_sv.write.mode("overwrite").partitionBy("Season").parquet("pga_partitioned_parquet")

25/11/15 21:34:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/11/15 21:34:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


## 8. Read in a partitioned parquet file

Now try reading in the partitioned parquet file you just created above. 

In [59]:
# Read partitioned parquet 
df_pga_partitioned = spark.read.parquet("pga_partitioned_parquet")

 
df_pga_partitioned.show()
 
# Maybe the schema
df_pga_partitioned.printSchema()

+-----+------+
|Value|Season|
+-----+------+
|   71|  2010|
|   77|  2010|
|   83|  2010|
|   54|  2010|
|  100|  2010|
|   63|  2010|
|   88|  2010|
|   64|  2010|
|   64|  2010|
|   92|  2010|
|   75|  2010|
|   54|  2010|
|   76|  2010|
|   94|  2010|
|   82|  2010|
|   85|  2010|
|   79|  2010|
|   89|  2010|
|   88|  2010|
|   91|  2010|
+-----+------+
only showing top 20 rows
root
 |-- Value: integer (nullable = true)
 |-- Season: integer (nullable = true)



## 9. Reading in a set of paritioned parquet files

Now try only reading Seasons 2010, 2011 and 2012.

In [60]:
df_pga_partitioned = spark.read.parquet("pga_partitioned_parquet")
df_pga_filtered = df_pga_partitioned.filter(df_pga_partitioned.Season.isin(2010, 2011, 2012))

df_pga_filtered.show()

# OR


+-----+------+
|Value|Season|
+-----+------+
|   71|  2010|
|   77|  2010|
|   83|  2010|
|   54|  2010|
|  100|  2010|
|   63|  2010|
|   88|  2010|
|   64|  2010|
|   64|  2010|
|   92|  2010|
|   75|  2010|
|   54|  2010|
|   76|  2010|
|   94|  2010|
|   82|  2010|
|   85|  2010|
|   79|  2010|
|   89|  2010|
|   88|  2010|
|   91|  2010|
+-----+------+
only showing top 20 rows


## 10. Create your own dataframe

Try creating your own dataframe below using PySparks *.createDataFrame* function. See if you can make one that contains 4 variables and at least 3 rows. 

Let's see how creative you can get on the content of the dataframe :)

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType

# Custom schema
bank_schema = StructType([
    StructField("account_num", IntegerType(), True),
    StructField("customer_name", StringType(), True),
    StructField("balance", DoubleType(), True),
    StructField("active", BooleanType(), True)
])

# Data
bank_data = [
    (657834321, "Abeiku James", 3250.75, True),
    (664567157, "Uwase Marie", 10500.10, True),
    (658934320, "Kamala Mutsie", 250.00, False)
]

# DataFrame
df_bank = spark.createDataFrame(bank_data, bank_schema)

df_bank.show()
df_bank.printSchema()


+-----------+-------------+-------+------+
|account_num|customer_name|balance|active|
+-----------+-------------+-------+------+
|  657834321| Abeiku James|3250.75|  true|
|  664567157|  Uwase Marie|10500.1|  true|
|  658934320|Kamala Mutsie|  250.0| false|
+-----------+-------------+-------+------+

root
 |-- account_num: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- balance: double (nullable = true)
 |-- active: boolean (nullable = true)



In [ ]:
# Data of customers
bank_data = [
    (657834321, "Abeiku James", 3250.75),
    (664567157, "Uwase Marie", 10500.10),
    (658934320, "Kamala Mutsie", 250.00),
    (664567157, "Ole Fomeh", 10500.10),
    (658934320, "Kamu Saile", 250.00)
]

# DataFrame 
df_bank = spark.createDataFrame(bank_data, ['account_num','customer_name','balance'])
df_bank.show()

+-----------+-------------+-------+
|account_num|customer_name|balance|
+-----------+-------------+-------+
|  657834321| Abeiku James|3250.75|
|  664567157|  Uwase Marie|10500.1|
|  658934320|Kamala Mutsie|  250.0|
|  664567157|    Ole Fomeh|10500.1|
|  658934320|   Kamu Saile|  250.0|
+-----------+-------------+-------+



In [62]:
spark.stop()

# **Part 2: Manipulating Data in DataFrames HW**


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [4]:
# PySpark instance

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession


# spark = SparkSession.builder.appName("Joseph_wrk2").getOrCreate()
spark = SparkSession.builder \
    .appName("Joseph_wrk2") \
    .master("local[*]") \
    .getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
appid = spark._jsc.sc().applicationId()
print("You are working with", cores, "core(s) on appid: ",appid)
spark

You are working with 1 core(s) on appid:  local-1763383033191


25/11/17 12:37:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [64]:
path = "data-sets/"
df_tweets = spark.read.csv(path + 'Rep_vs_Dem_tweets.csv', header=True, inferSchema=True)


## About this dataframe

Extracted tweets from all of the representatives (latest 200 as of May 17th 2018)

**Source:** https://www.kaggle.com/kapastor/democratvsrepublicantweets#ExtractedTweets.csv

Use either .show() or .toPandas() check out the first view rows of the dataframe to get an idea of what we are working with.

In [65]:
df_tweets.show()


+--------------------+-------------+--------------------+
|               Party|       Handle|               Tweet|
+--------------------+-------------+--------------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|
|Congress has allo...|         NULL|                NULL|
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
|            Democrat|RepDarrenSoto|RT @Vegalteno: Hu...|
|            Democrat|RepDarrenSoto|RT @EmgageActionF...|
|            Democrat|RepDarrenSoto|Hurricane Maria l...|
|            Democrat|RepDarrenSoto|RT @Tharryry: I a...|
|            Democrat|RepDarrenSoto|RT @HispanicCaucu...|
|            Democrat|RepDarrenSoto|RT @RepStephMurph...|
|            Democrat|RepDarrenSoto|RT @AllSaints_FL:...|
|            Democrat|RepDarrenSoto|.@realDonaldTrump...|
|            Democrat|RepDarrenSoto|Thank you to my m...|
|            D

**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [66]:
df_tweets.show(truncate=False)

+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party                                               |Handle       |Tweet                                                                                                                                       |
+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat                                            |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat                                            |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized 

**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [67]:
df_tweets.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [68]:
from pyspark.sql.functions import regexp_extract

# New column to extract "@LatinoLeader" for tracking
df_w_latino = df_tweets.withColumn(
    "latino_found",
    regexp_extract("Tweet", "@LatinoLeader", 0)
)

# Filter tweets w\ the handle
df_w_latino.filter(df_w_latino.latino_found != "").show(truncate=False)
 

+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|Party   |Handle       |Tweet                                                                                                                                       |latino_found |
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|Democrat|RepDarrenSoto|RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…|@LatinoLeader|
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+-------------+



## 2. Replace any value other than 'Democrate' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrate' or 'Republican' in the Part column. We are assuming that this is dirty data that needs to be cleaned up.

In [69]:
# Checking whether indeed the "Party" col is dirty with things except Demo.. and Repub..

df_tweets.groupBy("Party").count().show()

+--------------------+-----+
|               Party|count|
+--------------------+-----+
|        investigat…"|    1|
|Pruitt’s Chief Of...|    1|
|05/03/… https://t...|    1|
|NOTE that… https:...|    1|
|MC Lyte - Poor Ge...|    1|
|   Start- November 1|    1|
|To all who are ce...|    1|
|The Kennedy famil...|    1|
|Glad… https://t.c...|    1|
|The U.S. should b...|    1|
|   -1st: Rosa Kirgis|    1|
|             If you:|    1|
|            As a mom|    1|
|          @jmeacham |    1|
|    #SuicidePreven…"|    1|
|                 1…"|    1|
|https://t.co/XL5w...|    1|
|I’m sure you’re j...|    1|
|As… https://t.co/...|    1|
|      ☔ Venice Beach|    1|
+--------------------+-----+
only showing top 20 rows


In [70]:
from pyspark.sql.functions import when

# Cleaning
df_tweets_clean_party = df_tweets.withColumn(
    "Party",
    when(df_tweets.Party.isin("Democrat", "Republican"), df_tweets.Party)
    .otherwise("Other")
)

df_tweets_clean_party.show()


+--------+-------------+--------------------+
|   Party|       Handle|               Tweet|
+--------+-------------+--------------------+
|Democrat|RepDarrenSoto|Today, Senate Dem...|
|Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|Democrat|RepDarrenSoto|RT @NBCLatino: .@...|
|   Other|         NULL|                NULL|
|Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
|Democrat|RepDarrenSoto|RT @Vegalteno: Hu...|
|Democrat|RepDarrenSoto|RT @EmgageActionF...|
|Democrat|RepDarrenSoto|Hurricane Maria l...|
|Democrat|RepDarrenSoto|RT @Tharryry: I a...|
|Democrat|RepDarrenSoto|RT @HispanicCaucu...|
|Democrat|RepDarrenSoto|RT @RepStephMurph...|
|Democrat|RepDarrenSoto|RT @AllSaints_FL:...|
|Democrat|RepDarrenSoto|.@realDonaldTrump...|
|Democrat|RepDarrenSoto|Thank you to my m...|
|Democrat|RepDarrenSoto|We paid our respe...|
|   Other|         NULL|                NULL|
|Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|Democrat|RepDarrenSoto|Meet 12 incredibl...|
|Democrat|RepDarrenSoto|RT @wildli

In [71]:
# Confirming Cleanliness # Democrat # Republican # Other

df_tweets_clean_party.groupBy("Party").count().show()

+----------+-----+
|     Party|count|
+----------+-----+
|  Democrat|42068|
|     Other| 6029|
|Republican|44392|
+----------+-----+



## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

In [72]:
from pyspark.sql.functions import regexp_replace

df_tweets_no_links = df_tweets_clean_party.withColumn(
    "Tweet",
    regexp_replace("Tweet", r"https?://\S+", "")
)

df_tweets_no_links.show(truncate=False)


+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party   |Handle       |Tweet                                                                                                                                       |
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat|RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…                         |
|Democrat|RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|Democrat|RepDarrenSoto|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|Oth

## 4. Remove any leading or trailing white space in the tweet column

In [73]:
from pyspark.sql.functions import trim

df_tweets_trimmed = df_tweets_no_links.withColumn(
    "Tweet",
    trim("Tweet")
)

df_tweets_trimmed.show(truncate=False)


+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party   |Handle       |Tweet                                                                                                                                       |
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat|RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…                         |
|Democrat|RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|Democrat|RepDarrenSoto|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|Oth

## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [74]:
df_tweets_renamed = df_tweets_trimmed.withColumnRenamed("Party", "Dem_Rep")

df_tweets_renamed.show(truncate=False)


+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Dem_Rep |Handle       |Tweet                                                                                                                                       |
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat|RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…                         |
|Democrat|RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|Democrat|RepDarrenSoto|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|Oth

## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [75]:
from pyspark.sql.functions import concat_ws

# Party has been changed to Dem_Rep

df_tweets_concat_PH = df_tweets_renamed.withColumn(
    "Party_Handle",
    concat_ws(" - ", "Dem_Rep", "Handle")
)

# df_tweets_concat_PH.limit(10).toPandas()

df_tweets_concat_PH.select("Dem_Rep", "Handle", "Party_Handle").show(truncate=False)


+--------+-------------+------------------------+
|Dem_Rep |Handle       |Party_Handle            |
+--------+-------------+------------------------+
|Democrat|RepDarrenSoto|Democrat - RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat - RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat - RepDarrenSoto|
|Other   |NULL         |Other                   |
|Democrat|RepDarrenSoto|Democrat - RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat - RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat - RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat - RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat - RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat - RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat - RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat - RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat - RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat - RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat - RepDarrenSoto|
|Other   |NULL         |Other                   |
|Democrat|RepDarrenSoto|Democrat - RepDarrenSoto|


## Challenge Question

Let's imagine that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

In [76]:

from pyspark.sql.functions import expr

df_hashtags = df_tweets_concat_PH.withColumn(
    "hashtags",
    expr("filter(split(Tweet, ' '), x -> x rlike '^#[A-Za-z0-9_]+' )")
)



In [77]:
from pyspark.sql.functions import size, col

df_hashtags_nonempty = df_hashtags.filter(
    (col("hashtags").isNotNull()) &
    (size(col("hashtags")) > 0)
)


In [78]:
df_hashtags_nonempty.select("hashtags").show(truncate=False)


+-----------------------------------+
|hashtags                           |
+-----------------------------------+
|[#SaveTheInternet., #NetNeutrality]|
|[#NALCABPolicy2018.…]              |
|[#NetNeutrality]                   |
|[#Orlando]                         |
|[#HFAnow, #paidsickdays]           |
|[#classroomwhereithappens]         |
|[#FACT:]                           |
|[#ABetterDeal]                     |
|[#Science, #Sayfie]                |
|[#2020Census.]                     |
|[#9Investigates', #wftv]           |
|[#APAHM]                           |
|[#Insulin4all]                     |
|[#EmgagetheHill, #EmgageAction]    |
|[#Emgagethehill]                   |
|[#TPS, #Hondurans]                 |
|[#HurricaneMaria]                  |
|[#vamos4pr]                        |
|[#DemsAtWor…]                      |
|[#DemsAtWork]                      |
+-----------------------------------+
only showing top 20 rows


# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients first and last names, date of birth, and the dates of their first 3 visits. 

In [79]:
from pyspark.sql.types import *

md_office = [('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2') \
            ,('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2') \
            ,('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2') \
            ,('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2') \
            ,('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2') \
            ,('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2') \
            ,('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2') ]

df = spark.createDataFrame(md_office,['first_name','last_name','dob','visit1','visit2','visit3']) # schema=final_struc

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)

None


Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

In [80]:
from pyspark.sql.functions import to_date

df2 = df.withColumn("dob", to_date("dob", "yyyy-M-d")) \
        .withColumn("visit1", to_date("visit1", "yyyy-M-d")) \
        .withColumn("visit2", to_date("visit2", "yyyy-M-d")) \
        .withColumn("visit3", to_date("visit3", "yyyy-M-d"))

df2.printSchema()



root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- visit1: date (nullable = true)
 |-- visit2: date (nullable = true)
 |-- visit3: date (nullable = true)



## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

In [81]:
from pyspark.sql.functions import datediff, col, avg

# Diff between visits 
df_vst_diffs = df.withColumn("visit_1_to_2", datediff(col("visit2"), col("visit1"))) \
    .withColumn("visit_2_to_3", datediff(col("visit3"), col("visit2")))

 
print("Days between visits ")
df_vst_diffs.show()

Days between visits 
+----------+---------+----------+--------+---------+---------+------------+------------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|visit_1_to_2|visit_2_to_3|
+----------+---------+----------+--------+---------+---------+------------+------------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|         393|         392|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|         727|         364|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|         180|         393|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|         758|          27|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|         241|         607|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|         483|         425|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|         274|        -215|
+----------+---------+----------+--------+---------+---------+------------+------------+


In [82]:
# Average visits
avg_btwn_vst = df_vst_diffs.select(
    avg("visit_1_to_2").alias("avg_wait_visit_1_to_2"),
    avg("visit_2_to_3").alias("avg_wait_visit_2_to_3")
)

print("Average time between visits:")
avg_btwn_vst.show()

Average time between visits:
+---------------------+---------------------+
|avg_wait_visit_1_to_2|avg_wait_visit_2_to_3|
+---------------------+---------------------+
|   436.57142857142856|    284.7142857142857|
+---------------------+---------------------+



## 8. Can you calculate the age of each patient?

In [83]:
from pyspark.sql.functions import current_date, months_between, floor

df_age = df2.withColumn(
    "age_years",
    floor(months_between(current_date(), "dob") / 12)
)

df_age.select("first_name", "last_name", "dob", "age_years").show()


+----------+---------+----------+---------+
|first_name|last_name|       dob|age_years|
+----------+---------+----------+---------+
|  Mohammed|   Alfasy|1987-04-08|       38|
|     Marcy|Wellmaker|1986-04-08|       39|
|     Ginny|   Ginger|1986-07-10|       39|
|     Vijay| Doberson|1988-05-02|       37|
|     Orhan|  Gelicek|1987-05-11|       38|
|     Sarah|    Jones|1956-07-06|       69|
|      John|  Johnson|2017-10-12|        8|
+----------+---------+----------+---------+



## 9. Can you extract the month from the first visit column and call it "Month"?

In [84]:
from pyspark.sql.functions import month, col

 # Month column
df_with_month = df_age.withColumn("Month", month(col("visit1")))

 
df_with_month.select("first_name", "last_name", "visit1", "Month").show()

+----------+---------+----------+-----+
|first_name|last_name|    visit1|Month|
+----------+---------+----------+-----+
|  Mohammed|   Alfasy|2016-01-07|    1|
|     Marcy|Wellmaker|2015-01-07|    1|
|     Ginny|   Ginger|2014-08-07|    8|
|     Vijay| Doberson|2016-01-07|    1|
|     Orhan|  Gelicek|2016-05-07|    5|
|     Sarah|    Jones|2016-04-07|    4|
|      John|  Johnson|2018-01-02|    1|
+----------+---------+----------+-----+



### OR with Names like January, February etc.. 

In [85]:
from pyspark.sql.functions import date_format

df_month_names = df_with_month.withColumn(
    "Month",
    date_format("visit1", "MMMM")
)

df_month_names.select("first_name", "last_name", "visit1", "Month").show()

+----------+---------+----------+-------+
|first_name|last_name|    visit1|  Month|
+----------+---------+----------+-------+
|  Mohammed|   Alfasy|2016-01-07|January|
|     Marcy|Wellmaker|2015-01-07|January|
|     Ginny|   Ginger|2014-08-07| August|
|     Vijay| Doberson|2016-01-07|January|
|     Orhan|  Gelicek|2016-05-07|    May|
|     Sarah|    Jones|2016-04-07|  April|
|      John|  Johnson|2018-01-02|January|
+----------+---------+----------+-------+



## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

In [86]:
path = "data-sets/"
df_sales = spark.read.csv(path + 'supermarket_sales.csv', header=True, inferSchema=True)
 
 
df_sales.show(5, truncate=False)
df_sales.printSchema()

+-----------+------+---------+-------------+------+----------------------+----------+--------+-------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
|Invoice ID |Branch|City     |Customer type|Gender|Product line          |Unit price|Quantity|Tax 5% |Total   |Date     |Time               |Payment    |cogs  |gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+----------------------+----------+--------+-------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
|750-67-8428|A     |Yangon   |Member       |Female|Health and beauty     |74.69     |7       |26.1415|548.9715|1/5/2019 |2025-11-15 13:08:00|Ewallet    |522.83|4.761904762            |26.1415     |9.1   |
|226-31-3081|C     |Naypyitaw|Normal       |Female|Electronic accessories|15.28     |5       |3.82   |80.22   |3/8/2019 |2025-11-15 10:29:00|Cash       |76.4  |4.761904762         

## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

In [87]:
path = "data-sets/"
df_sales = spark.read.csv(path + 'supermarket_sales.csv', header=True, inferSchema=True)
 
df_sales.limit(5).toPandas()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,2025-11-15 13:08:00,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,2025-11-15 10:29:00,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,2025-11-15 13:23:00,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,2025-11-15 20:33:00,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,2025-11-15 10:37:00,Ewallet,604.17,4.761905,30.2085,5.3


In [88]:
df_sales.show(5, truncate=False)
df_sales.printSchema()

+-----------+------+---------+-------------+------+----------------------+----------+--------+-------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
|Invoice ID |Branch|City     |Customer type|Gender|Product line          |Unit price|Quantity|Tax 5% |Total   |Date     |Time               |Payment    |cogs  |gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+----------------------+----------+--------+-------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
|750-67-8428|A     |Yangon   |Member       |Female|Health and beauty     |74.69     |7       |26.1415|548.9715|1/5/2019 |2025-11-15 13:08:00|Ewallet    |522.83|4.761904762            |26.1415     |9.1   |
|226-31-3081|C     |Naypyitaw|Normal       |Female|Electronic accessories|15.28     |5       |3.82   |80.22   |3/8/2019 |2025-11-15 10:29:00|Cash       |76.4  |4.761904762         

### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

In [89]:
# the to_date() already imported

df_sales = df_sales.withColumn(
    "Date",
    to_date("Date", "M/d/yyyy")
)

df_sales.printSchema()
df_sales.show(5)


root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+----------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   T

### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

In [90]:
 # month() already imported

df_sales = df_sales.withColumn("Month", month("Date"))
df_sales.select("Date", "Month").show(10)
df_sales.printSchema()


+----------+-----+
|      Date|Month|
+----------+-----+
|2019-01-05|    1|
|2019-03-08|    3|
|2019-03-03|    3|
|2019-01-27|    1|
|2019-02-08|    2|
|2019-03-25|    3|
|2019-02-25|    2|
|2019-02-24|    2|
|2019-01-10|    1|
|2019-02-20|    2|
+----------+-----+
only showing top 10 rows
root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Month: intege

## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [91]:
from pyspark.sql.functions import *

values = [(5,'Epic. This is the best movie I have EVER seen'), \
          (4,'Pretty good, but I would have liked to seen better special effects'), \
          (3,'So so. Casting could have been improved'), \
          (5,'The most EPIC movie of the year! Casting was awesome. Special effects were so intense.'), \
          (4,'Solid but I would have liked to see more of the love story'), \
          (5,'THE BOMB!!!!!!!')]
reviews = spark.createDataFrame(values,['rating', 'review_txt'])

reviews.show(6,False)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|4     |Pretty good, but I would have liked to seen better special effects                    |
|3     |So so. Casting could have been improved                                               |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
|4     |Solid but I would have liked to see more of the love story                            |
|5     |THE BOMB!!!!!!!                                                                       |
+------+--------------------------------------------------------------------------------------+



## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the rview_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

In [92]:
df_cleaned = reviews.withColumn("cleaned_txt", 
    trim(lower(regexp_replace(col("review_txt"), r"[^\w\s]", "")))
)

# Split the cleaned
df_array = df_cleaned.withColumn("review_array", split(col("cleaned_txt"), " "))

 
df_array.select("rating", "review_array").show(truncate=False)


+------+---------------------------------------------------------------------------------------------------+
|rating|review_array                                                                                       |
+------+---------------------------------------------------------------------------------------------------+
|5     |[epic, this, is, the, best, movie, i, have, ever, seen]                                            |
|4     |[pretty, good, but, i, would, have, liked, to, seen, better, special, effects]                     |
|3     |[so, so, casting, could, have, been, improved]                                                     |
|5     |[the, most, epic, movie, of, the, year, casting, was, awesome, special, effects, were, so, intense]|
|4     |[solid, but, i, would, have, liked, to, see, more, of, the, love, story]                           |
|5     |[the, bomb]                                                                                        |
+------+-----------

## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

In [93]:
from pyspark.sql.functions import array_contains

reviews_epic = df_array.filter(array_contains("review_array", "epic"))

reviews_epic.select("review_txt", "review_array").show(truncate=False)


+--------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|review_txt                                                                            |review_array                                                                                       |
+--------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------+
|Epic. This is the best movie I have EVER seen                                         |[epic, this, is, the, best, movie, i, have, ever, seen]                                            |
|The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|[the, most, epic, movie, of, the, year, casting, was, awesome, special, effects, were, so, intense]|
+------------------------------------------------------

In [94]:
spark.stop()

# **Part 3: Joining and Appending DataFrames in PySpark HW**

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [2]:
# PySpark instance

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession


 
spark = SparkSession.builder \
    .appName("Joseph_wrk3") \
    .master("local[*]") \
    .getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
appid = spark._jsc.sc().applicationId()
print("You are working with", cores, "core(s) on appid: ",appid)
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/15 22:04:37 WARN Utils: Your hostname, redsteam-inspiron-3580, resolves to a loopback address: 127.0.1.1; using 10.6.248.103 instead (on interface wlp3s0)
25/11/15 22:04:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/15 22:04:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


You are working with 1 core(s) on appid:  local-1763244280606


## Read in the database

Let cotinue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [ ]:
import os

path = "data-sets/uw-madison-courses/"

df_list = []
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filename_list = filename.split(".")  
        df_name = filename_list[0]
        df = spark.read.csv(path+filename,inferSchema=True,header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + ' = df')
        
# For referencing
print("Full list of dfs:")
print(df_list)

Full list of dfs:
['course_offerings', 'grade_distributions', 'instructors', 'rooms', 'subjects', 'sections', 'courses', 'teachings', 'schedules', 'subject_memberships']


Now check the contents of a few of the dataframses that were read in above.

In [97]:
subjects.show(5)
rooms.show(5)
sections.show(5)
courses.show(5)
schedules.show(5)


+----+--------------------+------------+
|code|                name|abbreviation|
+----+--------------------+------------+
| 908|        Soil Science|    SOIL SCI|
| 350|             ENGLISH|     ENGLISH|
| 351|English as a Seco...|         ESL|
| 230|Human Development...|        HDFS|
| 352|             English|        ENGL|
+----+--------------------+------------+
only showing top 5 rows
+--------------------+-------------+---------+
|                uuid|facility_code|room_code|
+--------------------+-------------+---------+
|04368a56-c959-3e4...|   OFF CAMPUS|     null|
|2cc50da3-ef0e-357...|         0032|     0249|
|ebbf62b4-2ac3-356...|         0032|     B101|
|ed828265-475b-31b...|         0032|     0549|
|b277dc8e-bab1-3a1...|         0032|     0349|
+--------------------+-------------+---------+
only showing top 5 rows
+--------------------+--------------------+------------+------+--------------------+--------------------+
|                uuid|course_offering_uuid|section_typ

## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [4]:
result = (
    sections
    .join(rooms, sections.room_uuid == rooms.uuid, "inner")
    .join(course_offerings, sections.course_offering_uuid == course_offerings.uuid, "inner")
    .select(
        rooms.uuid.alias("room_uuid"),
        rooms.facility_code,
        rooms.room_code,
        course_offerings.term_code,
        course_offerings["name"].alias("course_name")
    )
)

result.show(50, truncate=False)


+------------------------------------+-------------+---------+---------+--------------------------+
|room_uuid                           |facility_code|room_code|term_code|course_name               |
+------------------------------------+-------------+---------+---------+--------------------------+
|04368a56-c959-3e4b-8b3d-f4cc3538fea5|OFF CAMPUS   |null     |1172     |Cooperative Education Prog|
|04368a56-c959-3e4b-8b3d-f4cc3538fea5|OFF CAMPUS   |null     |1172     |Cooperative Education Prog|
|04368a56-c959-3e4b-8b3d-f4cc3538fea5|OFF CAMPUS   |null     |1114     |Cooperative Education Prog|
|04368a56-c959-3e4b-8b3d-f4cc3538fea5|OFF CAMPUS   |null     |1104     |Cooperative Education Prog|
|04368a56-c959-3e4b-8b3d-f4cc3538fea5|OFF CAMPUS   |null     |1134     |Cooperative Education Prog|
|04368a56-c959-3e4b-8b3d-f4cc3538fea5|OFF CAMPUS   |null     |1134     |Cooperative Education Prog|
|04368a56-c959-3e4b-8b3d-f4cc3538fea5|OFF CAMPUS   |null     |1084     |Cooperative Education Prog|


## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [5]:
result.filter(rooms.facility_code == "0469").show(100, truncate=False)


+------------------------------------+-------------+---------+---------+--------------------------+
|room_uuid                           |facility_code|room_code|term_code|course_name               |
+------------------------------------+-------------+---------+---------+--------------------------+
|9759cb5f-a7d3-3d0c-b8b3-03910bd0153e|0469         |2441     |1152     |Fundamentals-Flute        |
|9759cb5f-a7d3-3d0c-b8b3-03910bd0153e|0469         |2441     |1092     |Fundamentals-Flute        |
|6af80b0b-b3e3-370a-925f-829869f007bc|0469         |4411     |1172     |Fundamentals-Flute        |
|9759cb5f-a7d3-3d0c-b8b3-03910bd0153e|0469         |2441     |1162     |Fundamentals-Flute        |
|9759cb5f-a7d3-3d0c-b8b3-03910bd0153e|0469         |2441     |1132     |Fundamentals-Flute        |
|50322d30-dd8f-3c65-9a75-78c5ff29d62c|0469         |2511     |1072     |Fundamentals-Flute        |
|9759cb5f-a7d3-3d0c-b8b3-03910bd0153e|0469         |2441     |1142     |Fundamentals-Flute        |


## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

In [100]:
from pyspark.sql.functions import count

result_q2 = (
    sections
    .join(subject_memberships,
          sections.course_offering_uuid == subject_memberships.course_offering_uuid,
          "inner")
    .join(rooms,
          sections.room_uuid == rooms.uuid,
          "inner")
    .groupBy("subject_code", "facility_code")
    .agg(count("*").alias("num_sections"))
    .orderBy("subject_code", "facility_code")
)

result_q2.show(30, truncate=False)



+------------+-------------+------------+
|subject_code|facility_code|num_sections|
+------------+-------------+------------+
|102         |0025         |42          |
|102         |0503         |30          |
|102         |1095         |211         |
|104         |0000         |1           |
|104         |0018         |11          |
|104         |0046         |182         |
|104         |0047         |13          |
|104         |0048         |30          |
|104         |0050         |74          |
|104         |0053         |11          |
|104         |0054         |8           |
|104         |0055         |17          |
|104         |0056         |54          |
|104         |0057         |45          |
|104         |0060         |4           |
|104         |0070         |4           |
|104         |0080         |3           |
|104         |0084         |1           |
|104         |0085         |1           |
|104         |0087A        |1           |
|104         |0106         |1     

## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

In [101]:
from pyspark.sql.functions import col, sum as sum_

# aggregate on  failures 
failures_by_course = (
    grade_distributions
    .groupBy("course_offering_uuid")
    .agg(sum_("f_count").alias("total_failures"))
)


In [102]:
# join for course names
hardest_classes = (
    failures_by_course
    .join(
        course_offerings,
        failures_by_course.course_offering_uuid == course_offerings["uuid"],
        "inner"
    )
    .select(
        course_offerings["name"].alias("course_name"), 
        col("total_failures")
    )
    .orderBy(col("total_failures").desc())
)

hardest_classes.show(20, truncate=False)

+----------------------------+--------------+
|course_name                 |total_failures|
+----------------------------+--------------+
|Calc--Functns of Variables  |72            |
|Animal Biology              |70            |
|Calculus&Analytic Geometry  |67            |
|Calculus&Analytic Geometry 1|64            |
|Calculus&Analytic Geometry  |63            |
|Calculus&Analytic Geometry  |59            |
|Calculus&Analytic Geometry  |58            |
|Calculus&Analytic Geometry 1|57            |
|Animal Biology              |56            |
|Animal Biology              |54            |
|Animal Biology              |53            |
|Calculus&Analytic Geometry  |53            |
|Calculus&Analytic Geometry  |52            |
|Animal Biology              |52            |
|Intro Organic Chemistry     |52            |
|Intro Organic Chemistry     |51            |
|Calculus                    |50            |
|Calculus&Analytic Geometry  |49            |
|Intro Organic Chemistry     |49  

## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [103]:
values = [('Monkey',10),('Monkay',36),('Mnky',123), \
          ('Elephant',48),('Elefant',16),('Ellafant',1), \
          ('Hippopotamus',48),('Hipopotamus',16),('Hippo',1)]
zoo = spark.createDataFrame(values,['Animal','age'])
zoo.show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkay| 36|
|        Mnky|123|
|    Elephant| 48|
|     Elefant| 16|
|    Ellafant|  1|
|Hippopotamus| 48|
| Hipopotamus| 16|
|       Hippo|  1|
+------------+---+



In [104]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# correct names
correct = ["Monkey", "Elephant", "Hippopotamus"]
correct_df = spark.createDataFrame([(c,) for c in correct], ["Correct"])

# cross join zoo with correct names
joined = zoo.crossJoin(correct_df)

# levenshtein distance
with_distance = joined.withColumn(
    "dist",
    F.levenshtein("Animal", "Correct")
)


In [105]:

# Partition per (Animal, age) and ordere by distance
w = Window.partitionBy("Animal", "age").orderBy("dist")


ranked = with_distance.withColumn("rn", F.row_number().over(w))

fixed = (
    ranked
    .filter(F.col("rn") == 1)            # keep only closest match
    .select("Animal", "age", "Correct", "dist")
    .withColumnRenamed("Correct", "CorrectedAnimal")
    .withColumnRenamed("dist", "Distance")
)

fixed.show(truncate=False)


+------------+---+---------------+--------+
|Animal      |age|CorrectedAnimal|Distance|
+------------+---+---------------+--------+
|Elefant     |16 |Elephant       |2       |
|Elephant    |48 |Elephant       |0       |
|Ellafant    |1  |Elephant       |3       |
|Hipopotamus |16 |Hippopotamus   |1       |
|Hippo       |1  |Monkey         |6       |
|Hippopotamus|48 |Hippopotamus   |0       |
|Mnky        |123|Monkey         |2       |
|Monkay      |36 |Monkey         |1       |
|Monkey      |10 |Monkey         |0       |
+------------+---+---------------+--------+



In [106]:
spark.stop()